In [2]:
import pandas as pd
import json
#Garbage Collector
import gc
import pickle

from google.colab import drive

drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
flightDataPath = 'gdrive/My Drive/FlightDelayPrediction/FlightData/'
weatherDataPath = 'gdrive/My Drive/FlightDelayPrediction/WeatherData/'
valid_airports = ('ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO')
years=['2016','2017']


master_df = pd.DataFrame()

# Generating Weather DataFrame

the corresponding airport id like 'JFK' has been appended to the weather data for each airport for each month in order to make the join between flight data and weather data easier. The same weather dataframe has been **saved as a pickle file** inorder to save computation during future runs of the program

In [0]:
weather_df = pd.DataFrame()

f = None
jsonOb = None


#Checks to see if previously generated pickle file is present, else generate new weather dictionary
try:
  pkl_file = open(weatherDataPath+'WeatherData.pkl','rb')
  weather_df = pickle.load(pkl_file)
  pkl_file.close()

except FileNotFoundError:

  pkl_file = open(weatherDataPath+'WeatherData.pkl','wb')

  i=False
  print('Computing!')

  for airport in valid_airports:
    for year in years:
      for month in range(1,12+1):
        windspeedKmph=[]
        FeelsLikeF=[]
        winddir16Point=[]
        FeelsLikeC=[]
        DewPointC=[]
        windspeedMiles=[]
        DewPointF=[]
        HeatIndexF=[]
        cloudcover=[]
        HeatIndexC=[]
        precipMM=[]
        weatherIconUrl=[]
        WindGustMiles=[]
        pressure=[]
        WindGustKmph=[]
        weatherDesc=[]
        visibility=[]
        weatherCode=[]
        tempC=[]
        tempF=[]
        WindChillF=[]
        WindChillC=[]
        winddirDegree=[]
        humidity=[]
        time=[]

        f = open(weatherDataPath + airport + '/' + year + '-' + str(month)+'.json')
        jsonOb = json.load(f)
        temp_df = pd.DataFrame(jsonOb['data']['weather'])
        hourly_data = temp_df['hourly']
        temp_df = temp_df.drop(columns=['hourly','astronomy'])

        temp_df = pd.concat([temp_df]*24,ignore_index=True)
        index =temp_df.columns.get_loc('date')
        temp_df = pd.DataFrame(sorted(temp_df.values,key=lambda x: x[index].split('-')[::-1]),columns=temp_df.columns)
        
        for i in range(len(hourly_data)):
          for j in range(len(hourly_data[0])):
            windspeedKmph.append(hourly_data[i][j]['windspeedKmph'])
            FeelsLikeF.append(hourly_data[i][j]['FeelsLikeF'])
            winddir16Point.append(hourly_data[i][j]['winddir16Point'])
            FeelsLikeC.append(hourly_data[i][j]['FeelsLikeC'])
            DewPointC.append(hourly_data[i][j]['DewPointC'])
            windspeedMiles.append(hourly_data[i][j]['windspeedMiles'])
            DewPointF.append(hourly_data[i][j]['DewPointF'])
            HeatIndexF.append(hourly_data[i][j]['HeatIndexF'])
            cloudcover.append(hourly_data[i][j]['cloudcover'])
            HeatIndexC.append(hourly_data[i][j]['HeatIndexC'])
            precipMM.append(hourly_data[i][j]['precipMM'])
            weatherIconUrl.append(hourly_data[i][j]['weatherIconUrl'])
            WindGustMiles.append(hourly_data[i][j]['WindGustMiles'])
            pressure.append(hourly_data[i][j]['pressure'])
            WindGustKmph.append(hourly_data[i][j]['WindGustKmph'])
            weatherDesc.append(hourly_data[i][j]['weatherDesc'])
            visibility.append(hourly_data[i][j]['visibility'])
            weatherCode.append(hourly_data[i][j]['weatherCode'])
            tempC.append(hourly_data[i][j]['tempC'])
            tempF.append(hourly_data[i][j]['tempF'])
            WindChillF.append(hourly_data[i][j]['WindChillF'])
            WindChillC.append(hourly_data[i][j]['WindChillC'])
            winddirDegree.append(hourly_data[i][j]['winddirDegree'])
            humidity.append(hourly_data[i][j]['humidity'])
            time.append(hourly_data[i][j]['time'])

        compiled_hourly = pd.DataFrame()
        compiled_hourly['windspeedKmph']=windspeedKmph
        compiled_hourly['FeelsLikeF']=FeelsLikeF
        compiled_hourly['FeelsLikeC']=FeelsLikeC
        compiled_hourly['winddir16Point']=winddir16Point
        compiled_hourly['DewPointC']=DewPointC
        compiled_hourly['windspeedMiles']=windspeedMiles
        compiled_hourly['DewPointF']=DewPointF
        compiled_hourly['HeatIndexF']=HeatIndexF
        compiled_hourly['cloudcover']=cloudcover
        compiled_hourly['HeatIndexC']=HeatIndexC
        compiled_hourly['precipMM']=precipMM
        compiled_hourly['weatherIconUrl']=weatherIconUrl
        compiled_hourly['WindGustMiles']=WindGustMiles
        compiled_hourly['pressure']=pressure
        compiled_hourly['WindGustKmph']=WindGustKmph
        compiled_hourly['weatherDesc']=weatherDesc
        compiled_hourly['visibility']=visibility
        compiled_hourly['weatherCode']=weatherCode
        compiled_hourly['tempC']=tempC
        compiled_hourly['tempF']=tempF
        compiled_hourly['WindChillF']=WindChillF
        compiled_hourly['WindChillC']=WindChillC
        compiled_hourly['winddirDegree']=winddirDegree
        compiled_hourly['humidity']=humidity
        compiled_hourly['time']=time
        temp_df = pd.concat([temp_df,compiled_hourly],axis=1)
        airportList = [airport for i in range(temp_df.shape[0])]
        temp_df['Airport']=airportList
        
        if i:
          weather_df = pd.concat([weather_df,temp_df],ignore_index=True)
        else:
          weather_df = temp_df
        i = True
        f.close()
  pickle.dump(weather_df,pkl_file)
  pkl_file.close()
      
  


In [54]:
weather_df.shape

(263160, 34)

In [39]:
weather_df.loc[40:50]

,mintempC,maxtempF,sunHour,mintempF,maxtempC,totalSnow_cm,date,uvIndex,windspeedKmph,FeelsLikeF,FeelsLikeC,winddir16Point,DewPointC,windspeedMiles,DewPointF,HeatIndexF,cloudcover,HeatIndexC,precipMM,weatherIconUrl,WindGustMiles,pressure,WindGustKmph,weatherDesc,visibility,weatherCode,tempC,tempF,WindChillF,WindChillC,winddirDegree,humidity,time,Airport
40,2,48,9.0,35,9,0.0,2016-01-02,0,12,41,5,WNW,-1,7,31,45,53,7,0.0,[{'value': 'http://cdn.worldweatheronline.net/...,9,1024,15,[{'value': 'Sunny'}],10,113,8,46,41,5,302,58,1600,ATL
41,2,48,9.0,35,9,0.0,2016-01-02,0,11,41,5,WNW,0,7,32,45,63,7,0.0,[{'value': 'http://cdn.worldweatheronline.net/...,9,1023,15,[{'value': 'Cloudy'}],10,119,8,46,41,5,303,59,1700,ATL
42,2,48,9.0,35,9,0.0,2016-01-02,0,11,42,5,WNW,0,7,32,45,73,7,0.0,[{'value': 'http://cdn.worldweatheronline.net/...,9,1023,15,[{'value': 'Cloudy'}],10,119,8,46,42,5,304,60,1800,ATL
43,2,48,9.0,35,9,0.0,2016-01-02,0,10,41,5,WNW,0,6,33,44,64,7,0.0,[{'value': 'http://cdn.worldweatheronline.net/...,9,1023,14,[{'value': 'Cloudy'}],10,119,7,45,41,5,302,64,1900,ATL
44,2,48,9.0,35,9,0.0,2016-01-02,0,9,40,4,WNW,1,6,33,43,55,6,0.0,[{'value': 'http://cdn.worldweatheronline.net/...,9,1023,14,[{'value': 'Partly cloudy'}],10,116,7,44,40,4,301,68,2000,ATL
45,2,48,9.0,35,9,0.0,2016-01-02,0,9,39,4,WNW,1,5,34,43,46,6,0.0,[{'value': 'http://cdn.worldweatheronline.net/...,8,1023,13,[{'value': 'Partly cloudy'}],10,116,6,43,39,4,299,71,2100,ATL
46,2,48,9.0,35,9,0.0,2016-01-02,0,9,38,4,WNW,1,6,34,42,47,6,0.0,[{'value': 'http://cdn.worldweatheronline.net/...,9,1023,14,[{'value': 'Partly cloudy'}],10,116,6,42,38,4,297,73,2200,ATL
47,2,48,9.0,35,9,0.0,2016-01-02,0,9,38,3,WNW,1,6,34,42,49,5,0.0,[{'value': 'http://cdn.worldweatheronline.net/...,9,1024,15,[{'value': 'Partly cloudy'}],10,116,5,42,38,3,295,74,2300,ATL
48,3,51,7.5,38,11,0.0,2016-01-03,0,10,37,3,WNW,1,6,34,41,50,5,0.0,[{'value': 'http://cdn.worldweatheronline.net/...,9,1024,15,[{'value': 'Partly cloudy'}],10,116,5,41,37,3,293,75,0,ATL
49,3,51,7.5,38,11,0.0,2016-01-03,0,10,37,3,WNW,1,6,34,41,56,5,0.0,[{'value': 'http://cdn.worldweatheronline.net/...,10,1023,16,[{'value': 'Partly cloudy'}],10,116,5,41,37,3,297,76,100,ATL


# Joining Flight Data of All Flights with the weather data of the departure airport of the day

In [55]:
try:
  joined_pkl = open(flightDataPath + 'JoinedData.pkl','rb')
  print('Loaded from File!')
  joined_df = pickle.load(joined_pkl)
  joined_pkl.close()

except FileNotFoundError:    
  print('Generating File!')
  joined_df = pd.DataFrame()
  temp_df = pd.DataFrame()

  i=False

  for year in years:
    for months in range(1,12+1):
      nearest_hour=[]
      temp_df = pd.read_csv(flightDataPath + year +'/' + str(months) + '.csv',low_memory=False)
      temp_df = temp_df.dropna(subset=['DepTime'])
      for i in range(len(temp_df)):
        time = temp_df.iloc[i]['DepTime']
        hr = time // 100
        mm = time % 100
        if mm >= 30:
          hr += 1
          if hr == 24:
            hr  = 23
        time = hr * 100
        if time == 0:
          nearest_hour.append('0')
        else:
          nearest_hour.append(str(int(time)))
      temp_df['NearestDepTime']=nearest_hour         
      temp_df = pd.merge(temp_df,weather_df,left_on=['FlightDate','Origin','NearestDepTime'],right_on=['date','Airport','time'])
      if i:
        joined_df = pd.concat([joined_df,temp_df],ignore_index=True)
      else:
        joined_df = temp_df
      i=True
  
  joined_pkl = open(flightDataPath + 'JoinedData.pkl','wb')
  pickle.dump(joined_df,joined_pkl)
  joined_pkl.close()

Generating File!


In [56]:
joined_df.shape

(1855994, 146)

In [59]:
joined_df.loc[1]['Airport']

'SEA'

In [62]:
joined_df.loc[1]['DepTime']


821.0